# Dynamic Tropopause Map

 http://www.met.nps.edu/~hmarcham/2012.html#DT
 The dynamic tropopause map contains
 DT (1.5-PVU surface)
    - potential temperature (shaded, K)
    - wind barbs (knots)
    - 925-850hPa layer-averaged cyclonic relative vorticity (black contours, every 0.5 x 10^-4 s^-1)



 http://apps.ecmwf.int/codes/grib/param-db
 #### Parameters from ECMWF
    - name = 'Potential temperature',      shortName = 'pt',     [K]
    - name = 'U component of wind',        shortName = 'u',      [m s^-1]
    - name = 'V component of wind',        shortName = 'v',      [m s^-1]
    - name = 'Vorticity (relative)',       shortName = 'vo',     [s^-1]



https://software.ecmwf.int/wiki/display/CKB/How+to+plot+GRIB+files+with+Python+and+matplotlib

In [1]:
import pygrib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap, shiftgrid
import numpy as np
from scipy import ndimage, signal, stats
%matplotlib inline     

In [2]:
### set the file name of your input GRIB file
#grib = 'DT_var.grib'
year = '2016'
mon  = '12'
date = '10' 
time = '0000'

gribPT = '../test_data/pv2000/param_pt_%s%s%s_%s.grib' % (year, mon, date, time)    #20161210_0000.grib'
gribUV = '../test_data/pv2000/param_uv_%s%s%s_%s.grib' % (year, mon, date, time)    #20161210_0000.grib'
gribVO = '../test_data/pl/param_vo_%s%s%s_%s.grib'     % (year, mon, date, time)    #20161210_0000.grib'

#grib = '../DT_var.grib'

### open GRIB file
grbsPT = pygrib.open(gribPT)         # potential temperature
grbsUV = pygrib.open(gribUV)         # U, V component of wind
grbsVO = pygrib.open(gribVO)         # Vorticity (relative)

#grbs = pygrib.open(grib)


In [3]:
grbPT = grbsPT.select()[0]           # finds the first grib message, if grbs.select(name='Maximum temperature') 
                                     # then first with matching name
grbUV = grbsUV.select()[0]
grbVO = grbsVO.select()[0]

#grb = grbs.select()[0]

In [4]:
print('pot. temp.:', grbPT.dataDate,grbPT.dataTime)                 # gives the date and forecast time
print('U, V component of wind:', grbUV.dataDate,grbUV.dataTime)
print('Vorticity (relative):', grbVO.dataDate,grbVO.dataTime)

pot. temp.: 20161210 0
U, V component of wind: 20161210 0
Vorticity (relative): 20161210 0


In [5]:
### get the values into python
# find the first grib message with a matching name: 

## Potential temperature
PT = grbsPT.select()[0]
PT = grbsPT.select(shortName = 'pt',typeOfLevel = 'potentialVorticity',level = 2000)[0]
PT = PT.values

#PT.shape
#print(PT.max(),PT.min())

In [6]:
#PT = grbs.select()[0]
#PT = grbs.select(shortName = 'pt',typeOfLevel = 'potentialVorticity',level = 2000)[0]
#PT = PT.values

In [7]:
## Wind 
# U component of wind
wind_u = grbsUV.select()[0]
wind_u = grbsUV.select(shortName = 'u',typeOfLevel = 'potentialVorticity',level = 2000)[0]
wind_u = wind_u.values
#print(wind_u.max(),wind_u.min())

# V component of wind
wind_v = grbsUV.select()[0]
wind_v = grbsUV.select(shortName = 'v',typeOfLevel = 'potentialVorticity',level = 2000)[0]
wind_v = wind_v.values
#print(wind_v.max(),wind_v.min())

In [8]:
#wind_u = grbs.select()[0]
#wind_u = grbs.select(shortName = 'u',typeOfLevel = 'potentialVorticity',level = 2000)[0]
#wind_u = wind_u.values

#wind_v = grbs.select()[0]
#wind_v = grbs.select(shortName = 'v',typeOfLevel = 'potentialVorticity',level = 2000)[0]
#wind_v = wind_v.values

In [9]:
## Vorticity (relative)
# 925 hPa
RV925 = grbsVO.select()[0]
RV925 = grbsVO.select(shortName = 'vo',typeOfLevel = 'isobaricInhPa', level = 925)[0]
RV925 = RV925.values
#print(RV925.max(),RV925.min())

# 900 hPa
RV900 = grbsVO.select()[0]
RV900 = grbsVO.select(shortName = 'vo',typeOfLevel = 'isobaricInhPa', level = 900)[0]
RV900 = RV900.values
#print(RV900.max(),RV900.min())

# 850 hPa
RV850 = grbsVO.select()[0]
RV850 = grbsVO.select(shortName = 'vo',typeOfLevel = 'isobaricInhPa', level = 850)[0]
RV850 = RV850.values
#print(RV850.max(),RV850.min())

In [10]:
#RV925 = grbs.select()[0]
#RV925 = grbs.select(shortName = 'vo',typeOfLevel = 'isobaricInhPa', level = 925)[0]
#RV925 = RV925.values

#RV900 = grbs.select()[0]
#RV900 = grbs.select(shortName = 'vo',typeOfLevel = 'isobaricInhPa', level = 900)[0]
#RV900 = RV900.values

#RV850 = grbs.select()[0]
#RV850 = grbs.select(shortName = 'vo',typeOfLevel = 'isobaricInhPa', level = 850)[0]
#RV850 = RV850.values

In [11]:
### calculating 925-850 hPa layer-averaged cyclonic relative vorticity (every 0.5 x10^-4 s-1)
# all three layers divided by number of layers
rel_vort = (RV925+RV900+RV850)/3


data = stats.shapiro(rel_vort)      # Perform the Shapiro-Wilk test for normality. The Shapiro-Wilk test tests the 
                                    # null hypothesis that the data was drawn from a normal distribution.
                                    # (W,p-value): W: The test statistic, p-value for hypothesis test
                                    # if p-value is < W than null hypothesis is rejected --> not normally distributed
print(data)

(0.8308833837509155, 0.0)


/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/scipy/stats/morestats.py:1326: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [12]:
### Latitudes, Longitudes and shiftgrid
# get the latitudes and longitudes of the grid
latPT,lonPT = grbPT.latlons()           # Set the names of the latitude and longitude variables in your input GRIB 
                                        # file 
latUV,lonUV = grbUV.latlons()
latVO,lonVO = grbVO.latlons()

#print(latUV.max(),latUV.min(), lonUV.max(), lonUV.min())
#print(latUV.shape, lonUV.shape)

In [13]:
#lat,lon = grb.latlons()

In [14]:
lonsPT = lonPT[0,:]                     #use only one row
lonsUV = lonUV[0,:]
lonsVO = lonVO[0,:]

PT,lonsPT = shiftgrid(180.,PT,lonsPT,start=False)          #shift all values from >180 deg
wind_u,lonsUV = shiftgrid(180.,wind_u,lonsUV,start=False)
rel_vort,lonsVO = shiftgrid(180.,rel_vort,lonsVO,start=False)    #shift all values from >180 deg


latsPT = latPT[:,0]                   # use only first column
latsUV = latUV[:,0]
latsVO = latVO[:,0]


#print(lonsPT.shape,latsPT.shape,PT.shape)
#print(lonsUV.shape,latsUV.shape,wind_u.shape)
#print(lonsUV.shape,latsUV.shape,wind_v.shape)
#print(lonsVO.shape,latsVO.shape,rel_vort.shape)

In [15]:
#lons = lon[0,:]
#PT,lons = shiftgrid(180.,PT,lons,start=False)
#lats = lat[:,0]


In [16]:
### Wind
# use only every 20, 35 value from the wind
UVlats = np.zeros((64,),dtype =float)
UVlons = np.zeros((74,),dtype =float)
u_wind = np.zeros((64,2560), dtype = float)
v_wind = np.zeros((64,2560), dtype = float)
u_wind2 = np.zeros((64,74),dtype = float)
v_wind2 = np.zeros((64,74),dtype = float)

    
#numrows = len(wind_u)    
#numcols = len(wind_u[0]) 
#print(numrows,numcols)

for i in range(0,64):
    UVlats[i] = latsUV[i*20]
    u_wind[i,:] = wind_u[i*20,:]
    v_wind[i,:] = wind_v[i*20,:]

for k in range(0,74):
    UVlons[k] = lonsUV[k*35]   
    u_wind2[:,k] = u_wind[:,k*35]
    v_wind2[:,k] = v_wind[:,k*35]


#print(UVlats.shape,UVlons.shape,u_wind2.shape,v_wind2.shape)


### Plotting data on a map (Example Gallery) https://matplotlib.org/basemap/users/examples.html



In [17]:
# Plotting data on a map (Example Gallery) https://matplotlib.org/basemap/users/examples.html
m = Basemap(projection='merc', \
            llcrnrlon=-60., urcrnrlon=50., \
            llcrnrlat=30.,urcrnrlat=75., \
            resolution='f')

#m = Basemap(projection='merc', \
#            llcrnrlon=-160., urcrnrlon=-60., \
#            llcrnrlat=20.,urcrnrlat=60., \
#            resolution='c')

#m = Basemap(projection='cyl', \
#            llcrnrlon=-180., urcrnrlon=180., \
#            llcrnrlat=latsPT.min(),urcrnrlat=latsPT.max(), \
#            resolution='c')


lonsPT,latsPT = np.meshgrid(lonsPT,latsPT)            #creates a grid with values of the size of PT
lonsVO,latsVO = np.meshgrid(lonsVO,latsVO)
UVlons,UVlats = np.meshgrid(UVlons,UVlats)


plonsPT, platsPT = m(lonsPT,latsPT)
plonsVO, platsVO = m(lonsVO,latsVO)
plonsUV, platsUV = m(UVlons,UVlats)

#print(plonsPT.shape,platsPT.shape,PT.shape)
#print(plonsVO.shape,platsVO.shape,rel_vort.shape)
#print(plonsUV.shape,platsUV.shape,u_wind2.shape)


In [18]:
#lons,lats = np.meshgrid(lons,lats)
#plons, plats = m(lons,lats)


In [19]:
### define colors for colorbar
champ = 255
no1 = np.array([231,231,231])/champ
no2 = np.array([201,201,201])/champ
no3 = np.array([171,171,171])/champ
no4 = np.array([140,140,140])/champ
no5 = np.array([110,110,110])/champ
no6 = np.array([102,81,224])/champ
no7 = np.array([105,115,224])/champ
no8 = np.array([103,134,255])/champ
no9 = np.array([132,159,254])/champ
no10= np.array([178,202,255])/champ
no11= np.array([217,241,255])/champ
no12= np.array([255,209,177])/champ
no13= np.array([255,165,133])/champ
no14= np.array([255,118,86])/champ
no15= np.array([255,62,12])/champ
no16= np.array([255,0,0])/champ
no17= np.array([244,0,146])/champ
no18= np.array([171,171,171])/champ
no19= np.array([201,201,201])/champ
no20= np.array([231,231,231])/champ
no21= np.array([255,255,255])/champ



### PLOT FIGURE
fig = plt.figure(figsize=(15,12.5))
ax = fig.add_subplot(1,1,1)



### Draw Latitude Lines
m.drawparallels(np.arange(-90.,120.,20.),labels=[1,0,0,0],fontsize=10,linewidth=0.2)


### Draw Longitude Lines
m.drawmeridians(np.arange(-180.,180.,20.),labels=[0,0,0,1],fontsize=10,linewidth=0.2)


### Draw Map
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()


### Plot contour lines for pot. temp and fill
levels = np.arange(258,390,6)
cmap = colors.ListedColormap([no1, no2, no3, no4, no5, no6, no7, no8, no9, no10, \
                              no11, no12, no13, no14, no15, no16, no17, no18, no19, no20, \
                             no21])
norm = colors.BoundaryNorm(boundaries = levels, ncolors=cmap.N)
cs = m.contourf(plonsPT,platsPT,PT,levels,norm=norm,cmap=cmap)
#cs = m.contourf(plons,plats,PT,levels,norm=norm,cmap=cmap)
#cs = m.contourf(plonsUV,platsUV,wind_u,shading='flat',cmap = plt.cm.hot, norm=colors.LogNorm())
#cs = m.contourf(plonsUV,platsUV,wind_v,shading='flat',cmap = plt.cm.hot, norm=colors.LogNorm())
#cs = m.contourf(plonsVO,platsVO,rel_vort,shading='flat',cmap = plt.cm.hot)#, norm=colors.LogNorm())
#cs = m.contourf(plons,plats,rel_vort,shading='flat',cmap = plt.cm.hot)#, norm=colors.LogNorm())



### Plot contour lines for layer averaged rel. vort 925-850 hPa
thickness = np.arange(0.5*10**(-4),3.*10**(-4),.5*10**(-4))
#
CVO = m.contour(plonsVO,platsVO,rel_vort,thickness,colors='k')
#CVO = m.contour(plons,plats,data)#,thickness)#,colors='k')
plt.clabel(CVO, fontsize = 10, inline = 1, fmt = '%1.4f')

### plot wind barbs
plt.barbs(plonsUV,platsUV,u_wind2,v_wind2,barbcolor='darkslategray')


### Add Colorbar
cbaxes = fig.add_axes([0.14, 0.05, 0.75, 0.03]) 
cbar = plt.colorbar(cs,orientation='horizontal',cax = cbaxes)#, cax = cbaxes)#, shrink=0.5)
cbar.ax.set_xlabel('potential vorticity')

### Title
#fig.suptitle('%s%s%s/%s' % (year, mon, date, time), fontsize=14, fontweight='bold') 
#ax.set_title('Dynamic Tropopause Map at 2 PVU, \n \
#wind barbs (m s$^{-1}$), \n \
#potential temperature (K, shaded according to colorbar), \n \
#925-850-hPa layer-averaged cyclonic relative vorticity (black contours, every 0.5 x 10$^{-4}$ s$^{-1}$), \n')

### Save
#plt.savefig('../synoptic_figs/DynTropo_%s%s%s_%s.png' % (year, mon, date,time))
#plt.savefig('dyn_tropo.png')     # Set the output file name

plt.show()